In [1]:
import argparse
import json
import os

In [2]:
import tensorflow as tf
print(tf.__version__)

1.15.0


In [3]:
# just a temporary workaround
# pass this code to the setup.py file of the final module
import sys
_ROOT_DIR = '{0}/gcp/cbidmltsf'.format(os.getenv("HOME"))
sys.path.append(_ROOT_DIR)

In [4]:
from dplstm.data import make_input_fn
from dplstm.model import DPLSTM

In [5]:
# build parameters dictionary for interactive execution
# in batch execution, this dictionary comes from parsed cli arguments

parameters = {
    'model_dir': 'lstm_46',
    'learning_rate': 0.01,
    'num_epochs': 20,  # NOT REQUIRED IN DISTRIBUTED MODE, IT IS OVERRIDDEN BY MAX_TRAIN_STEPS
    'max_train_steps': 1000,
    'train_batch_size': 32,
    'eval_interval': 300,
    'keep_checkpoint_max': 3,
    'eval_steps': None,
    'eval_batch_size': 2**16,
    'start_delay_secs': 600,
    'throttle_secs': 600,
    'train_data_path': '{0}/data/tfrecord/train.tfrecord'.format(_ROOT_DIR),
    'eval_data_path': '{0}/data/tfrecord/val.tfrecord'.format(_ROOT_DIR),
    'test_data_path': '{0}/data/tfrecord/test.tfrecord'.format(_ROOT_DIR),    
}

In [6]:
# support for log files
import logging

# advanced numerical operations
import numpy as np

import tensorflow as tf

_LOG_PATH = '{0}/logs'.format(_ROOT_DIR)
logging.basicConfig(filename='{}/test.log'.format(_LOG_PATH),
                    level=logging.INFO,
                    format='%(asctime)s - %(name)s - %(threadName)s -  %(levelname)s - %(message)s')

logging.info('Logging to {}/test.log.'.format(_LOG_PATH))

# ToDo: pass a complex dictionary as a starting point for the LSTM network chromosome
'''
    Deep/Parallel LSTM network chromosome includes:
    m_hour: [24, 12, 8, etc], tau=1, implement this variable later and work now with m_hour=24
    m_day: [7, 14, etc], tau=7, implement this variable later and work now with m_day=7
    m_week: [4, 8, 12, etc], tau=168, implement this variable later and work now with m_week=4
    hidden_hour:
    hidden_day:
    hidden_week:
    levels_hour:
    levels_day:
    levels_week:
'''

_LOG_PATH = '{0}/logs'.format(_ROOT_DIR)
logging.basicConfig(filename='{}/test.log'.format(_LOG_PATH),
                    level=logging.INFO,
                    format='%(asctime)s - %(name)s - %(threadName)s -  %(levelname)s - %(message)s')

logging.info('Logging to {}/test.log.'.format(_LOG_PATH))

In [7]:
# remove main function and transfer everything to first-level code cells
tf.logging.set_verbosity(tf.logging.INFO)

In [8]:
def time_series_forecaster(features, labels, mode):
    # instantiate network topology from the corresponding class
    forecaster_topology = DPLSTM()

    # ToDo: global_step might be moved to TRAIN scope
    global_step = tf.train.get_global_step()

    # call operator to forecaster_topology, over features
    forecast = forecaster_topology(features)

    # predictions are stored in a dictionary for further use at inference stage
    predictions = {
        "forecast": forecast
    }

    # CHANGE MODEL FUNCTION STRUCTURE ACCORDING TO GILLARD'S ARCHITECTURE

    # Estimator in TRAIN or EVAL mode
    if mode == tf.estimator.ModeKeys.TRAIN or mode == tf.estimator.ModeKeys.EVAL:
        # use labels and predictions to define training loss and evaluation loss
        # generate summaries for TensorBoard
        with tf.name_scope('loss'):
            mean_squared_error = tf.losses.mean_squared_error(
                labels=labels, predictions=forecast, scope='loss')
            tf.summary.scalar('loss', mean_squared_error)

        with tf.name_scope('val_loss'):
            val_loss = tf.metrics.mean_squared_error(
                labels=labels, predictions=forecast, name='mse')
            tf.summary.scalar('val_loss', val_loss[1])

        # this is only required for PREDICT mode
        prediction_hooks = None

        # Estimator in TRAIN mode ONLY
        if mode == tf.estimator.ModeKeys.TRAIN:
            # This is needed for batch normalization, but has no effect otherwise
            update_ops = tf.get_collection(key=tf.GraphKeys.UPDATE_OPS)
            with tf.control_dependencies(control_inputs=update_ops):
                train_op = tf.contrib.layers.optimize_loss(
                    loss=mean_squared_error,
                    global_step=global_step,
                    learning_rate=parameters['learning_rate'],
                    optimizer="Adam")
            # Create a hook to print acc, loss & global step every 100 iter
            train_hook_list = []
            train_tensors_log = {'val_loss': val_loss[1],
                                 'loss': mean_squared_error,
                                 'global_step': global_step}
            train_hook_list.append(tf.train.LoggingTensorHook(
                tensors=train_tensors_log, every_n_iter=100))

            predictions = None  # this is not required in TRAIN mode
            loss = mean_squared_error
            eval_metric_ops = None
            training_hooks = train_hook_list
            evaluation_hooks = None

        else:  # Estimator in EVAL mode ONLY
            loss = mean_squared_error
            train_op = None
            training_hooks = None
            eval_metric_ops = {'val_loss': val_loss}
            evaluation_hooks = None

    # Estimator in PREDICT mode ONLY
    else:
        loss = None
        train_op = None
        eval_metric_ops = None
        training_hooks = None
        evaluation_hooks = None
        prediction_hooks = None  # this might change as we are in PREDICT mode

    return tf.estimator.EstimatorSpec(
        mode=mode,
        predictions=predictions,
        loss=loss,
        train_op=train_op,
        eval_metric_ops=eval_metric_ops,
        # export_outputs=not_used_yet (for TensorFlow Serving, redirected from predictions if omitted)
        # training_chief_hooks=not_used_yet
        # ToDo: verify use of training_hooks
        # temporarily disable training hooks
        # training_hooks=training_hooks,
        training_hooks=training_hooks,
        # scaffold=not_used_yet
        evaluation_hooks=evaluation_hooks,
        prediction_hooks=prediction_hooks
    )

# ToDo: evaluate the convenience of packaging execution in a tf.app
# in the meantime, run the estimator outside tf.app package
# tf.logging.set_verbosity(tf.logging.INFO)

In [9]:
# ensure file writer cache is clear for TensorBoard events file
tf.summary.FileWriterCache.clear()

In [10]:
# parameters required for RunConfig()
# _EVAL_INTERVAL = 300  # how often checkpoints are written out, given in seconds
# _KEEP_CHECKPOINT_MAX = 3  # how many checkpoints to keep
# _MAX_TRAIN_STEPS = 16000
# _TRAIN_BATCH_SIZE = 2**5
# _EVAL_STEPS = None  # if None, evaluate on entire dataset
# _EVAL_BATCH_SIZE = 2**16
# _START_DELAY_SECONDS = 600
# _THROTTLE_SECONDS = 600

In [11]:
# instantiate base estimator class for custom model function
tsf_estimator = tf.estimator.Estimator(
    model_fn=time_series_forecaster,
    # no parameters passed at this point
    # params=hparams,
    # parameters passed in original model include: train_data_path, batch_size, augment, train_steps
    config=tf.estimator.RunConfig(
        save_checkpoints_secs=parameters['eval_interval'],
        keep_checkpoint_max=parameters['keep_checkpoint_max']
    ),
    model_dir='{0}/dplstm/{1}'.format(_ROOT_DIR, parameters['model_dir']))

INFO:tensorflow:Using config: {'_num_worker_replicas': 1, '_protocol': None, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_global_id_in_cluster': 0, '_save_checkpoints_steps': None, '_experimental_max_worker_delay_secs': None, '_train_distribute': None, '_task_id': 0, '_keep_checkpoint_max': 3, '_master': '', '_evaluation_master': '', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7effdd5f7ef0>, '_eval_distribute': None, '_experimental_distribute': None, '_model_dir': '/home/jupyter/gcp/cbidmltsf/dplstm/lstm_46', '_save_checkpoints_secs': 300, '_num_ps_replicas': 0, '_task_type': 'worker', '_session_creation_timeout_secs': 7200, '_save_summary_steps': 100, '_tf_random_seed': None, '_is_chief': True, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_service': None, '_device_fn': None}


In [35]:
# Set estimator's train_spec to use train_input_fn and train for so many steps
train_spec = tf.estimator.TrainSpec(
    input_fn=make_input_fn(
        tfrecord_path=parameters['train_data_path'],
        batch_size=parameters['train_batch_size'],
        mode=tf.estimator.ModeKeys.TRAIN
    ),
    max_steps=parameters['max_train_steps'])

In [13]:
# ToDo: activate TensorFlow Serving for distributed inference
# Create exporter that uses serving_input_fn to create saved_model for serving
# exporter = tf.estimator.LatestExporter(
#     name="exporter",
#     serving_input_receiver_fn=serving_input_fn)

In [33]:
# Set estimator's eval_spec to use eval_input_fn and export saved_model
eval_spec = tf.estimator.EvalSpec(
    input_fn=make_input_fn(
        tfrecord_path=parameters['eval_data_path'],
        batch_size=parameters['eval_batch_size'],
        mode=tf.estimator.ModeKeys.EVAL
    ),
    steps=parameters['eval_steps'],  # use None to evaluate on the entire dataset
    exporters=exporter,
    start_delay_secs=parameters['start_delay_secs'],  # delay first evaluation
    throttle_secs=parameters['throttle_secs']  # evaluate at a different rate (usually longer) than checkpoint
)

In [36]:
# Run train_and_evaluate loop
tf.estimator.train_and_evaluate(
    estimator=tsf_estimator,
    train_spec=train_spec,
    eval_spec=eval_spec)

INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps None or save_checkpoints_secs 300.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /home/jupyter/gcp/cbidmltsf/dplstm/lstm_46/model.ckpt-900
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 900 into /home/jupyter/gcp/cbidmltsf/dplstm/lstm_46/model.ckpt.
INFO:tensorflow:loss = 0.005017233, step = 901
INFO:tensorflow:global_step = 901, loss = 0.005017233, val_loss = 0.005017233
INFO:tensorflow:Saving checkpoints for 1000 into /home/jupyter/gcp/cbidmltsf/dplstm/lstm_46

({'global_step': 1000, 'loss': 0.00549519, 'val_loss': 0.00549519},
 [b'/home/jupyter/gcp/cbidmltsf/dplstm/lstm_46/export/exporter/1575917715'])

In [16]:
# interactive design of serving_input_fn

In [23]:
feature_placeholders_demo = {
    # this example from https://stackoverflow.com
    # /questions/48510264/in-tensorflow-for-serving-a-model-what-does-the-serving-input-function-supposed
    # receives 5 scalar values as features
    # let's dive into types :-)
    'var1': tf.placeholder(tf.float32, [None]),
    'var2': tf.placeholder(tf.float32, [None]),
    'var3': tf.placeholder(tf.float32, [None]),
    'var4': tf.placeholder(tf.float32, [None]),
    'var5': tf.placeholder(tf.float32, [None])
}

In [24]:
# test the newly created dictionary
feature_placeholders_demo['var1']

<tf.Tensor 'Placeholder_5:0' shape=(?,) dtype=float32>

In [25]:
# features are expanded-dimensions versions of placeholders from [None] to [None, 1]
features_demo = {
    key: tf.expand_dims(tensor, -1)
    for key, tensor in feature_placeholders_demo.items()
}

In [26]:
features_demo

{'var1': <tf.Tensor 'ExpandDims_6:0' shape=(?, 1) dtype=float32>,
 'var2': <tf.Tensor 'ExpandDims_5:0' shape=(?, 1) dtype=float32>,
 'var3': <tf.Tensor 'ExpandDims_7:0' shape=(?, 1) dtype=float32>,
 'var4': <tf.Tensor 'ExpandDims_9:0' shape=(?, 1) dtype=float32>,
 'var5': <tf.Tensor 'ExpandDims_8:0' shape=(?, 1) dtype=float32>}

In [ ]:
# now the customized input function for serving predictions on dplstm
# target dictionary is {'hourly': [?, 24, 1], 'daily': [?, 7, 1], 'weekly': [?, 4, 1]}

In [27]:
feature_placeholders = {
    'hourly': tf.placeholder(tf.float32, [None, 24]),
    'daily': tf.placeholder(tf.float32, [None, 7]),
    'weekly': tf.placeholder(tf.float32, [None, 4])
}

In [28]:
features = {
    key: tf.expand_dims(tensor, -1)
    for key, tensor in feature_placeholders.items()
}

In [29]:
features

{'daily': <tf.Tensor 'ExpandDims_10:0' shape=(?, 7, 1) dtype=float32>,
 'hourly': <tf.Tensor 'ExpandDims_11:0' shape=(?, 24, 1) dtype=float32>,
 'weekly': <tf.Tensor 'ExpandDims_12:0' shape=(?, 4, 1) dtype=float32>}

In [31]:
def serving_input_fn():

    feature_placeholders = {
    'hourly': tf.placeholder(tf.float32, [None, 24]),
    'daily': tf.placeholder(tf.float32, [None, 7]),
    'weekly': tf.placeholder(tf.float32, [None, 4])
    }
    
    features = {
    key: tf.expand_dims(tensor, -1)
    for key, tensor in feature_placeholders.items()
    }
    
    return tf.estimator.export.ServingInputReceiver(features = features, receiver_tensors = feature_placeholders)

In [32]:
# create exporter that uses serving_input_fn to create saved_model for serving
exporter = tf.estimator.LatestExporter(
    name = "exporter", 
    serving_input_receiver_fn = serving_input_fn)

In [37]:
# predict based on checkpoints and plot results with Bokeh

In [38]:
# Anaconda Interactive Visualization
from bokeh.plotting import figure
from bokeh.plotting import output_file, save

In [39]:
# persistence for the scaler located in $DATA/scalers
from sklearn.externals import joblib

# reload scaler fitted model here
scaler = joblib.load('{0}/data/scalers/ci_LSTM_scaler.save'.format(_ROOT_DIR))

/usr/local/lib/python3.5/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator MinMaxScaler from version 0.19.1 when using version 0.21.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [40]:
# build predictions using the trained model
# ToDo: use TensorFlow Serving instead of the custom estimator
# pred = list(tsf_estimator.predict(input_fn=test_input_fn))
pred = list(
    tsf_estimator.predict(
        input_fn=make_input_fn(
            tfrecord_path=parameters['test_data_path'],
            batch_size=32,
            mode=tf.estimator.ModeKeys.PREDICT
        )
    )
)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /home/jupyter/gcp/cbidmltsf/dplstm/lstm_46/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [41]:
len(pred)

566

In [43]:
# let's look at a predictions subset
pred[:10]

[{'forecast': array([0.4926325], dtype=float32)},
 {'forecast': array([0.54299], dtype=float32)},
 {'forecast': array([0.5770303], dtype=float32)},
 {'forecast': array([0.58423465], dtype=float32)},
 {'forecast': array([0.6234549], dtype=float32)},
 {'forecast': array([0.6371721], dtype=float32)},
 {'forecast': array([0.6415772], dtype=float32)},
 {'forecast': array([0.6419458], dtype=float32)},
 {'forecast': array([0.6172749], dtype=float32)},
 {'forecast': array([0.56820893], dtype=float32)}]

In [44]:
# move to array and re-scale
pred = [p['forecast'][0] for p in pred]
pred = np.asarray(pred)
pred_ci = scaler.inverse_transform(pred.reshape(-1, 1))
pred_ci = np.squeeze(pred_ci)

In [51]:
pred_ci[:10]

array([4.056008 , 4.470618 , 4.7508826, 4.810199 , 5.1331124, 5.2460504,
       5.2823186, 5.2853537, 5.0822296, 4.6782537], dtype=float32)

In [46]:
# ToDo: get ytarget_test array from test.tfrecord dataset to perform the following operation
# temporarily get the array from disk
y_test = np.load('{0}/data/arrays/y_test.npy'.format(_ROOT_DIR))
n = 0  # first step ahead
ytarget_test = y_test[:, n]
ytarget_test = ytarget_test.reshape(ytarget_test.shape[0], 1)

actual_ci = scaler.inverse_transform(ytarget_test)
actual_ci = np.squeeze(actual_ci)

In [48]:
_EQUIPMENT = 'CPE04105'

In [49]:
ci_predictions_fig = figure(title='Predicted Current Imbalance for ' + _EQUIPMENT,
                            background_fill_color='#E8DDCB',
                            plot_width=1800, plot_height=450, x_axis_type='datetime')

# ToDo: yts_test array is required to get timestamps for plot, then wire it now and get it from TFRecord later...
yts_test = np.load('{0}/data/arrays/yts_test.npy'.format(_ROOT_DIR), allow_pickle=True)

ci_predictions_fig.line(yts_test[:, n],
                        actual_ci, line_color='red',
                        line_width=1, alpha=0.7, legend='Actual')

ci_predictions_fig.line(yts_test[:, n],
                        pred_ci, line_color='blue',
                        line_width=1, alpha=0.7, legend='Predicted')

ci_predictions_fig.legend.location = "top_right"
ci_predictions_fig.legend.background_fill_color = "darkgrey"

ci_predictions_fig.xaxis.axis_label = 'Timestamp'
ci_predictions_fig.yaxis.axis_label = 'Current Imbalance [%]'

# output_file('{0}/plots/'.format(_ROOT_DIR) + '{:03d}'.format(n) + '.html', title=_EQUIPMENT)
output_file('{0}/plots/'.format(_ROOT_DIR) + '{}'.format(parameters['model_dir']) + '.html', title=_EQUIPMENT)
save(ci_predictions_fig)

'/home/jupyter/gcp/cbidmltsf/plots/lstm_46.html'